In this step, we import the essential Python libraries used for data manipulation, visualisation, and machine learning throughout the project.

- pandas:  Data manipulation and analysis

- numpy: Numerical operations

- matplotlib and seaborn: Data visualisation

In [2]:
import pandas as pd
import numpy as np # for mathematical operations
import seaborn as sns # for data visualization
import matplotlib.pyplot as plt # for plotting graphs
%matplotlib inline
import warnings
warnings.filterwarnings('ignore') # to ignore warnings

For this problem, three CSV files are provided: train, test, and sample submission.

- Train File: Used to train the model. It contains all the independent variables (features) as well as the target variable.

- Test File: Contains only the independent variables. The trained model will be applied to this dataset to predict the target variable.

- Sample Submission File: Provides the required format for submitting the model’s predictions.

In [6]:
train_data = pd.read_csv('/Users/hamza/Desktop/projects/Loan_Prediction/Loan_Prediction/datasets/train_ctrUa4K.csv')
test_data = pd.read_csv('/Users/hamza/Desktop/projects/Loan_Prediction/Loan_Prediction/datasets/test_lAUu6dG.csv')

# Create a copy of the original data
train_data_copy = train_data.copy()
test_data_copy = test_data.copy()